# Alessandro Tofani
# Matricola 844145

Importo le librerie che userò per l'analisi dati,tra cui pandas, numpy, matplotlib

In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


# 1. Data input 

Importo i dati in:
- train_data = dati per fare allenare il modello.  Contiene l'autore e il subreddit in cui hanno scritto;
- target = label dei dati. 0 se sono uomini, 1 se sono donne

In [2]:
train_data = pd.read_csv("../input/datamining2020/train_data.csv", encoding="utf8")

Visualizzo le prime righe del dataset

In [3]:
train_data.head()

,author,subreddit,created_utc,body
0,Shamus_Aran,mylittlepony,1.388534e+09,I don't think we'd get nearly as much fanficti...
1,Riddance,sex,1.388534e+09,"Thanks. I made it up, that's how I got over my..."
2,Secret_Wizard,DragonsDogma,1.388534e+09,Are you sure you aren't confusing Cyclops (the...
3,Penultimatum,malefashionadvice,1.388534e+09,dont do this to me bro
4,7-SE7EN-7,todayilearned,1.388534e+09,That's what we do when we can't find a mate


In [4]:
train_data.author.unique().shape

(5000,)

Importo il dataset che contiene l'output

In [5]:
target = pd.read_csv("../input/datamining2020/train_target.csv")

In [6]:
target.head()

,author,gender
0,RedThunder90,0
1,Lirkmor,1
2,In0chi,0
3,ProjectGrudge,0
4,TehTurtleHermit,0


# 2. Feature Extraction

Seleziono le features che sono importanti per il modello:
- Subreddits alle quale l'autore ha partecipato
- Testo del messaggio
- Gender dell'autore

Estraggo tali informazioni dai rispettivi dataset e le organizzo in matrici. 

In [7]:
subreddits = train_data.subreddit.unique()
subreddits_map = pd.Series(index=subreddits, data=arange(subreddits.shape[0]))

Uso la libreria sparse che mi permette di lavorare con matrici sparse, ovvero matrici in cui la maggior parte degli elimenti è nulla. 

In [8]:
from scipy import sparse
from scipy.sparse import hstack

Definisco la funzione per estrarre le features. Estraggo il subreddit a cui l'utente ha partecipato. Per fare ciò, la funzione ritorna 1 se l'utente ha partecipato ad un subreddit, 0 altrimenti. 

Riga = autore

Colonna = subreddit

In [9]:
def extract_features(group):
    group_subreddits = group['subreddit'].values
    idxs = subreddits_map[group_subreddits].values
    v = sparse.dok_matrix((1, subreddits.shape[0]))
    for idx in idxs:
        if not np.isnan(idx):
            v[0, idx] = 1
    return v.tocsr()

extract_features(train_data[train_data.author=='RedThunder90'])

<1x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [10]:
features_dict = {}

for author, group in train_data.groupby('author'):
    features_dict[author] = extract_features(group)

Per occupare meno spazio in memoria e dato che la matrice contiene tanti zeri, la converto in sparse. 

In [11]:
X = sparse.vstack([features_dict[author] for author in target.author])
X

<5000x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 49152 stored elements in Compressed Sparse Row format>

In [12]:
y = target.gender

Definisco la funzione che estrae il testo del commento dell'autore. Quindi la funzione associa il testo all'autore. 

In [13]:
def extract_text(group):
    group_text = group['body'].values
    return " ".join(map(str,group_text))

extract_text(train_data[train_data.author=='RedThunder90'])

'I still prefer to buy foods either grown locally or where animals are treated better, but this definitely has me looking at organic food differently.'

In [14]:
text_dict = {}

for author, group in train_data.groupby('author'):
    text_dict[author] = extract_text(group)

In [15]:
author_text = [text_dict[author] for author in target.author]
author_text[0][:100]

'I still prefer to buy foods either grown locally or where animals are treated better, but this defin'

### CountVectorizer
Faccio la vectorization di author_text, cioè assegno un token ad ogni parola, e conto i token assegnati. Alla fine manterrò solo le parole che ricorrono di più o che hanno un peso maggiore. Gli elementi della matrice che estraggo sono [autore, parola]. Come prima, anche questa matrice sarà una matrice sparse. 

Tramite il parametro max_features specifico il massimo numero di parole che il CountVectorizer estrae. Dopo diverse prove, per avere una buona accuracy e consentire che il programma funzionasse correttamente, ho impostato 20000 come limite. 

Altresì bisogna rimuovere le parole che non portare alcuna informazione utili, come ad esempio le parole brevi. Per farlo uso:
- stop_words: non inserisce le stop words della lingua inglese;

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, ENGLISH_STOP_WORDS

pattern ='(?u)\\b[A-Za-z]{3,}'

stop_words = set(list(ENGLISH_STOP_WORDS) + ['test'])

cv = CountVectorizer(token_pattern=pattern, stop_words=stop_words, max_features = 20000)

C = cv.fit_transform(author_text)

tfidf = TfidfTransformer()

X_train = tfidf.fit_transform(C)

print(X_train.shape)

(5000, 20000)


Si è ottenuto:
- X_train: matrice con i valori delle parole usato dagli utenti;
- y: matrice con le label, cioè i gender;
- X: matrice contenti i subreddit

Unisco le due matrici contenenti i dati per formare X_total, che avrà 5000 righe e 23468 colonne, di cui le prime 3468 si riferiscono ai subreddit, mentre le altre alle parole selezionate. 

In [17]:
X_total = sparse.hstack([X,X_train])
X_total

<5000x23468 sparse matrix of type '<class 'numpy.float64'>'
	with 1934913 stored elements in COOrdinate format>

In [18]:
X_total = X_total.toarray()

# 3. Model selection

## Multinomial naive Bayes

Inizialmente ho usato il Multinomial Naive Bayes. Sebbene riuscissi ad ottenere una accuracy in sample superiore a 0.92, non sono riuscito a raggiungere una accuracy out of sample soddisfacente.

In [19]:
#from sklearn.naive_bayes import MultinomialNB
#from sklearn import model_selection

#NB = MultinomialNB()
#NB.fit(X_training, y)

#alphas = np.logspace(-3, 3, 20)
#scores = []
#cv1 = model_selection.KFold(n_splits=10, shuffle=True, random_state=0)

#for alpha in alphas:
#    NB.alpha = alpha
#    s = model_selection.cross_val_score(NB, X_training, 
#                                        y, cv=cv1)
#    scores.append(s.mean())

In [20]:
#plot(alphas, scores)
#plt.xscale('log')

In [21]:
#best_alpha = alphas[np.argmax(scores)]
#NB = MultinomialNB(alpha=best_alpha)
#NB.fit(X_total, y)
#print("Score: %s" % NB.score(X_total,y))

## Neural Network

Come secondo modello ho provato la rete neurale, implementandola con keras. Sono riuscito a raggiungere una accuracy in sample decisamente migliore di quella ottenuta con il Naive Bayes, di circa 0.98, impostando 20epochs e 200 di batch size. Prima di arrivare a tali parametri, ho fatto diverse submission per verificare l'accuracy out of sample, e verificare che il modello riuscisse a descrivere bene i dati senza overfittare. 

Come funzione di attivazione ho optato per la ReLU negli hidden layer, per evitare la saturazione del segnale, mentre nel layer di output ho optato per la sigmoide, avendo così come output la probabilità ell'input di appartenere ad uno dei due gender. 

Come loss function ho optato per la bynary_crossentropy, dato che il problema è di classificazione. 

Ho anche provato ad introdurre sia la regolarizzazione L1 che L2, ma non apportava alcun miglioramento al modello. 

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [23]:
def multilayer_perceptron(nr_hidden=20):
    # create model
    n_input = 23468
    model = Sequential()
    model.add(Dense(nr_hidden, input_dim=n_input, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [24]:
model = multilayer_perceptron()
h = model.fit(X_total, y, epochs=20, batch_size=500)

Epoch 1/20
10/10 [==============================] - 0s 35ms/step - loss: 0.6406 - accuracy: 0.7096
Epoch 2/20
10/10 [==============================] - 0s 36ms/step - loss: 0.5613 - accuracy: 0.7336
Epoch 3/20
10/10 [==============================] - 0s 35ms/step - loss: 0.5067 - accuracy: 0.7556
Epoch 4/20
10/10 [==============================] - 0s 35ms/step - loss: 0.4514 - accuracy: 0.8276
Epoch 5/20
10/10 [==============================] - 0s 38ms/step - loss: 0.4007 - accuracy: 0.8830
Epoch 6/20
10/10 [==============================] - 0s 35ms/step - loss: 0.3585 - accuracy: 0.8970
Epoch 7/20
10/10 [==============================] - 0s 41ms/step - loss: 0.3242 - accuracy: 0.9138
Epoch 8/20
10/10 [==============================] - 0s 36ms/step - loss: 0.2951 - accuracy: 0.9280
Epoch 9/20
10/10 [==============================] - 0s 36ms/step - loss: 0.2705 - accuracy: 0.9322
Epoch 10/20
10/10 [==============================] - 0s 39ms/step - loss: 0.2492 - accuracy: 0.9408
Epoch 11/

# 4. Prepare the solution

Ho importato i dati per predirre gli output sul test set. Il test set non contiene i valori del gender degli utenti. 

In [25]:
test_data = pd.read_csv("../input/datamining2020/test_data.csv", encoding="utf8")

In [26]:
features_dict = {}

for author, group in test_data.groupby('author'):
    features_dict[author] = extract_features(group)

In [27]:
X_test = sparse.vstack([features_dict[author] for author in test_data.author.unique()])
X_test

<15000x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 144898 stored elements in Compressed Sparse Row format>

In [28]:
text_dict = {}

for author, group in test_data.groupby('author'):
    text_dict[author] = extract_text(group)

In [29]:
author_text = [text_dict[author] for author in test_data.author.unique()]
author_text[0][:100]

"I hadn't ever heard of them before joining this subreddit. They're not really a big thing in the US,"

In [30]:
C = cv.transform(author_text)

X_test1 = tfidf.fit_transform(C)

In [31]:
X_test_total = sparse.hstack([X_test,X_test1])
X_test_total = X_test_total.toarray()

In [32]:
X_test_total.shape

(15000, 23468)

Vado a predirre qual è la probabilità di avere gender 1, usando il modello allenato sul trainig set. 

In [33]:
y_pred = model.predict(X_test_total)

In [34]:
y_pred = y_pred.reshape(15000,)

In [35]:
solution = pd.DataFrame({"author":test_data.author.unique(), "gender":y_pred})
solution.head(10)

,author,gender
0,ejchristian86,0.998543
1,ZenDragon,0.000665
2,savoytruffle,0.002261
3,hentercenter,0.009024
4,rick-o-suave,0.104667
5,olivermihoff,0.028586
6,Cleriesse,0.784628
7,murderer_of_death,0.000113
8,SpiralSoul,0.015997
9,IRideVelociraptors,0.077240


In [36]:
# solution.to_csv("solution.csv", index=False)